In [1]:
import numpy as np
import time

In [2]:
with open("input_09.txt", "r") as fh:
    diskmap = fh.read().strip()

In [3]:
# Pretty print of list
def print_diskmap(l):
    s = ""
    for li in l:
        if li is None:
            s += "."
        else:
            s += str(li)
    print(s)

In [4]:
# Convert blocks to list, with None for empty space
blocks = []
current_block = 0
for i, s in enumerate(diskmap):
    d = int(s)
    if not i % 2:
        # Block
        for j in range(d):
            blocks.append(current_block)
        current_block += 1
    else:
        # Empty space
        for j in range(d):
            blocks.append(None)

In [5]:
# 9a
t0 = time.time()
empty_idx = [idx for idx, b in enumerate(blocks) if b is None]
idx_empty_idx = 0
blocks_9a = blocks[:]
for i, b in enumerate(blocks[::-1]):
    if b is not None:
        blocks_9a[empty_idx[idx_empty_idx]] = b
        blocks_9a[len(blocks) - i - 1] = None
        idx_empty_idx += 1
    # Stop when there are no more empty idx left, or when the empty idx's are all right of the blocks
    if idx_empty_idx >= len(empty_idx) or empty_idx[idx_empty_idx] >= len(blocks) - i - 1:
        break
checksum = sum([b*i for i, b in enumerate(blocks_9a) if b is not None])
print(f"9a: {checksum} (in: {time.time() - t0} sec)")

9a: 6395800119709 (in: 0.022747516632080078 sec)


In [6]:
# 9b
t0 = time.time()
# Create dict with index of empty space, and number of sequenctial spaces after that (incl.)
empty_lengths = {empty_idx[0]: 1} # Dict: k=index, v=number of empty spots
cur_empty_val = empty_idx[0]
for i, ei in enumerate(empty_idx[1:]):
    if ei == empty_idx[i] + 1:
        empty_lengths[cur_empty_val] += 1
    else:
        cur_empty_val = ei
        empty_lengths[cur_empty_val] = 1

# Create dict of files: k=block id of file, v=(index of first block, number of blocks in file) (note: inefficient)
files = {} 
for sb in set(blocks):
    if sb is not None:
        files[sb] = (blocks.index(sb), blocks.count(sb))

# Go through empty space, and try to fill with files backwards
added_files = {}
for empty_start_idx, empty_space in empty_lengths.items():
    # Try to fill empty space with one or more files from right to left
    remaining_empty_space = empty_space
    latest_empty_start_idx = empty_start_idx
    scan_files_again = True # True if there may be space for another file
    while scan_files_again:
        scan_files_again = False
        temp_moved_keys = []
        for k in list(files.keys())[::-1]:
            if remaining_empty_space >= files[k][1] and files[k][0] > latest_empty_start_idx:
                # File fits
                temp_moved_keys.append(k)
                added_files[k] = (latest_empty_start_idx, files[k][1])
                remaining_empty_space -= files[k][1]
                latest_empty_start_idx += files[k][1]
                # Check if more empty space here
                if remaining_empty_space > 0:
                    scan_files_again = True
                break
        # House keeping: remove files that are already moved
        for k in temp_moved_keys:
            files.pop(k)

# Re-create blocks list
blocks_9b = [None for b in blocks]
all_files = files|added_files
for k in all_files.keys():
    for offset in range(all_files[k][1]):
        blocks_9b[all_files[k][0]+offset] = k

checksum = sum([b*i for i, b in enumerate(blocks_9b) if b is not None])
print(f"9b: {checksum} (in: {time.time() - t0} sec)")

9b: 6418529470362 (in: 10.524753093719482 sec)
